# Overview

Run everything in order to generate the images and accuracy estimates. The model can be modified by specifying `k_sigma` (immediately after Modeling heading).

Information about the data can be found here: [NHANES 2017-2020 Survey Homepage](https://wwwn.cdc.gov/Nchs/Nhanes/continuousnhanes/default.aspx?cycle=2017-2020)

An outline of this notebook is as follows:
- Function Definitions: defining the functions that compute the distributions/binomial parameters
- Data Collection and Preprocessing: pulls the data, cleans it, and does a train-test split
- Train-Test Split Visualization: plots the distribution of the train and test data
- Modeling: where the binomial distributions are computed and modified

In [ ]:
# Load necessary libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import binom

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
lirio_brand_colors = ['#F06923','#0A4150','#FEC010','#4A707A','#B9561D','#C9D4D7','#F59105']

# Function Definitions

In [ ]:
# Functions that define probabilities and sample sizes and compute binomial coefficients
def probabilities(k,s):
    if k <= 0:
        return s
    elif 1 - (s / k) <= 0:
        return 0.00001
    else:
        return 1 - (s / k)
    
def samples(k,p):
    if k <= 0:
        return 1
    elif p <= 0:
        return 0
    else:
        return k / p
    
def binomial_coefficient(n,p,k):
    if k > np.round(n,0):
        return 0.
    elif p >= 1:
        if np.round(n,0) == k:
            return 1.
        else:
            return 0.
    elif p <= 0:
        if k <= 0:
            return 1.
        else:
            return 0.
    else:
        return binom(n,k)*np.power(p,k)*np.power(1. - p,n-k)

In [ ]:
def generate_binomial_parameters(k_sigma_vals,response):
    
    # Split k and sigma values
    k_vals = [ksig[0] for ksig in k_sigma_vals]
    sigma_vals = [ksig[1] for ksig in k_sigma_vals]
    
    # Compute probabilities
    p_vals = [probabilities(i[0],i[1]) for i in k_sigma_vals]
    
    # Compute samples
    n_vals = [samples(i[0],i[1]) for i in zip(k_vals,p_vals)]
#     n_vals = [np.round(n,0) for n in n_vals]
    
    # Compute weights
    binom_coefs = [[binomial_coefficient(n,p,k) for p,n in zip(p_vals,n_vals)] for k in k_vals]
    binom_coefs.append([1.] * len(k_vals))
    w_response = [response['prob'].loc[np.round(k,0)] for k in k_vals] + [1.]
    reg = LinearRegression(fit_intercept = False).fit(binom_coefs,w_response)
    
    return n_vals,p_vals,reg.coef_

In [ ]:
def generate_distribution(n_vals,p_vals,weights):
    dfg = pd.DataFrame(
        data = zip(
            n_vals,
            p_vals
        ),
        columns = ['sample','success'],
        index = [f'k{i}' for i in range(1,1+len(n_vals))]
    )

    binomial_df = pd.DataFrame(
        data = range(22),
        columns = ['k']
    )

    for i in dfg.index:
        binomial_df[i] = None
        n = dfg['sample'].loc[i]
        p = dfg['success'].loc[i]
        binomial_df[i] = [binomial_coefficient(n,p,k) for k in np.linspace(0,21,22)]

    binomial_df['prob'] = binomial_df.apply(lambda x: np.dot(weights,x[1:]), axis = 1)

    return binomial_df

In [ ]:
def generate_new_scale(
    p_vals,
    n_vals,
    k_sigma_vals,
    response
):
    
    k_vals = [ksig[0] for ksig in k_sigma_vals]
    
    # Compute weights
    binom_coefs = [[binomial_coefficient(n,p,k) for p,n in zip(p_vals,n_vals)] for k in k_vals]
    binom_coefs.append([1.] * len(k_vals))
    w_response = [response['prob'].loc[np.round(k,0)] for k in k_vals] + [1.]
    reg = LinearRegression(fit_intercept = False).fit(binom_coefs,w_response)
    
    return reg.coef_

In [ ]:
def modify_probabilities(
    p_vals,
    n_vals,
    weights,
    k_sigma_vals,
    response,
    factor_scale = 0.1,
    factor_direction = 1,
    variable = 'gender_male',
    new_scale = False,
):
    
    zero_p_vals = [max(min((1-factor_direction * factor_scale)*(p-0.5)+0.5,1),0) for p in p_vals]
    one_p_vals = [max(min((1+factor_direction * factor_scale)*(p-0.5)+0.5,1),0) for p in p_vals]
    
    print(f"0: {', '.join([f'$p_{i}^f={np.round(zero_p_vals[i],3)}$'for i in range(len(zero_p_vals))])}")
    print(f"1: {', '.join([f'$p_{i}^m={np.round(one_p_vals[i],3)}$'for i in range(len(one_p_vals))])}")
    
    original_binomial_df = generate_distribution(n_vals,p_vals,weights)
    original_binomial_df['num_meals_not_home'] = original_binomial_df['k']
    original_binomial_df[variable] = 'Original'
    original_binomial_df['data'] = 'original'
    
    if new_scale:
        zero_weights = generate_new_scale(
            zero_p_vals,
            n_vals,
            k_sigma_vals,
            response
        )
    else:
        zero_weights = weights
    
    zero_binomial_df = generate_distribution(n_vals,zero_p_vals,zero_weights)
    zero_binomial_df[variable] = feature_dict[variable][0]
    zero_binomial_df['num_meals_not_home'] = zero_binomial_df['k']
    zero_binomial_df['data'] = 'modified'
    
    if new_scale:
        one_weights = generate_new_scale(
            one_p_vals,
            n_vals,
            k_sigma_vals,
            response
        )
    else:
        one_weights = weights
        
    one_binomial_df = generate_distribution(n_vals,one_p_vals,one_weights)
    one_binomial_df[variable] = feature_dict[variable][1]
    one_binomial_df['num_meals_not_home'] = one_binomial_df['k']
    one_binomial_df['data'] = 'modified'
    
    return original_binomial_df,zero_binomial_df,one_binomial_df

# Data Collection and Preprocessing

## Load Data

1. Pull data from website
2. Label columns with readable names

In [ ]:
# Demographic Data Column Name Meanings
demo_names_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/search/variablelist.aspx?Component=Demographics&Cycle=2017-2020'

# Demographic Data
demo_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DEMO.XPT'

# Survey Data Column Name Meanings
survey_names_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/search/variablelist.aspx?Component=Questionnaire&Cycle=2017-2020'

# Survey Data
survey_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DBQ.XPT'

In [ ]:
demo_names = pd.read_html(demo_names_url)[0]
survey_names = pd.read_html(survey_names_url)[0]
demo = pd.read_sas(demo_url)
survey = pd.read_sas(survey_url)

In [ ]:
demo['SEQN'] = demo['SEQN'].astype(int)
survey['SEQN'] = survey['SEQN'].astype(int)

In [ ]:
demo_names_dict = demo_names.set_index('Variable Name')['Variable Description'].to_dict()
demo_names_dict['SEQN'] = 'SEQN'

survey_names_dict = survey_names.set_index('Variable Name')['Variable Description'].to_dict()
survey_names_dict['SEQN'] = 'SEQN'

In [ ]:
demo.columns = [demo_names_dict[i] for i in demo.columns]
survey.columns = [survey_names_dict[i] for i in survey.columns]

## Data Preprocessing

1. Pick columns of interest
2. Drop missing values from demographic data
3. Correct missing values in number of fast food meals
4. Drop missing values from survey data
5. Merge demographic and survey data
6. Drop values out of acceptable range
7. Convert coded values to readable values
8. One-hot encode categorical values

In [ ]:
demo_small = demo[[
    'SEQN',
    'Gender of the participant.',
    'Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age.',
    'Recode of reported race and Hispanic origin information, with Non-Hispanic Asian Category',
    'What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?',
    'Marital status',
    'A ratio of family income to poverty guidelines.'
]].copy(deep = True).dropna()
demo_small.columns = [
    'SEQN',
    'gender',
    'age',
    'race',
    'education',
    'marital_status',
    'income'
]

for col in demo_small.columns:
    demo_small[col] = demo_small[col].astype(int)

In [ ]:
survey_small = survey[[
    'SEQN',
    "Next I'm going to ask you about meals. By meal, I mean breakfast, lunch and dinner. During the past 7 days, how many meals {did you/did SP} get that were prepared away from home in places such as restaurants, fast food places, food stands, grocery stores, or from vending machines? {Please do not include meals provided as part of the school lunch or school breakfast./Please do not include meals provided as part of the community programs you reported earlier.}",
    'How many of those meals {did you/did SP} get from a fast-food or pizza place?',
    'Some grocery stores sell "ready to eat" foods such as salads, soups, chicken, sandwiches and cooked vegetables in their salad bars and deli counters. During the past 30 days, how often did {you/SP} eat "ready to eat" foods from the grocery store? Please do not include sliced meat or cheese you buy for sandwiches and frozen or canned foods.',
    'During the past 30 days, how often did you {SP} eat frozen meals or frozen pizzas? Here are some examples of frozen meals and frozen pizzas.'
]].copy(deep = True)#.dropna()
survey_small.columns = [
    'SEQN',
    'num_meals_not_home',
    'num_meals_fast_food',
    'num_ready_foods_30_days',
    'num_frozen_foods_30_days'
]

# Drop rows with missing `num_meals_not_home`
survey_small = survey_small[~survey_small['num_meals_not_home'].isna()]
survey_small['num_meals_not_home'] = survey_small['num_meals_not_home'].astype(int)

# If `num_meals_not_home` is 0 and `num_meals_fast_food` is missing, replace missing value with 0
survey_small.loc[
    survey_small[
        (survey_small['num_meals_fast_food'].isna()) &
        (survey_small['num_meals_not_home'] == 0)
    ].index,
    'num_meals_fast_food'
] = 0

# Drop rows with missing values
survey_small.dropna(inplace = True)

# Converting these to integers, which `num_meals_not_home` and `num_meals_fast_food` may not be
for col in survey_small.columns:
    survey_small[col] = survey_small[col].astype(int)

In [ ]:
df = pd.merge(
    demo_small,
    survey_small,
    on='SEQN'
).set_index('SEQN')

In [ ]:
# Drop the unknowns education
df.drop(df[df['education'].isin([7,9])].index, inplace = True)

# Drop unknown marital status
df.drop(df[df['marital_status'].isin([77,99])].index, inplace = True)

# Drop out of range number of meals eaten, not prepared at home
df.drop(df[df['num_meals_not_home'] > 21].index, inplace = True)
df.drop(df[df['num_meals_fast_food'] > df['num_meals_not_home']].index, inplace = True)

# Drop meals that are too high (I think it means unknown anyway)
df.drop(df[df['num_ready_foods_30_days'] > 90].index, inplace = True)

# Drop meals that are too high
df.drop(df[df['num_frozen_foods_30_days'] > 90].index, inplace = True)

In [ ]:
# Feature value names from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DEMO.htm
gender_dict = {
    1: 'male',
    2: 'female'
}

race_dict = {
    1: 'Minority', #'Mexican American',
    2: 'Minority', #'Other Hispanic',
    3: 'Majority', #'Non-Hispanic White',
    4: 'Minority', #'Non-Hispanic Black',
    6: 'Minority', #'Non-Hispanic Asian',
    7: 'Minority', #'Other Race'
}

education_dict = {
    1: 'Less than 9th grade',
    2: '9-11th grade (Includes 12th grade with no diploma)',
    3: 'High school graduate/GED or equivalent',
    4: 'Some college or AA degree',
    5: 'College graduate or above',
    7: 'unknown',
    9: 'unknown'
}

marital_dict = {
    1: 'Married', #'Married/Living with Partner',
    2: 'Single', #'Widowed/Divorced/Separated',
    3: 'Single', #'Never married',
    77: 'unknown',
    99: 'unknown'
}

In [ ]:
# Replace coded values with their meaning
df.gender = df.gender.map(gender_dict)
df.race = df.race.map(race_dict)
df.marital_status = df.marital_status.map(marital_dict)
# Note: there are no 'unknown' values
# We will keep education as an ordinal value

In [ ]:
# One-hot encode categorical variables
df_dummies = pd.get_dummies(df, drop_first = True)

In [ ]:
# Dictionary to convert from dummy variables to their meaning
feature_dict = {
    'marital_status_Single':{
        0:'Married',
        1:'Single'
    },
    'race_Minority':{
        0: 'Majority',
        1: 'Minority'
    },
    'gender_male':{
        0: 'Female',
        1: 'Male'
    }
}

## Train-Test Split

In [ ]:
# Split into training and test sets
X_train, X_test, _, _ = train_test_split(
    df_dummies,
    df_dummies['num_meals_not_home'],
    test_size=0.33,
    random_state=330396
)

# Generate train distribution
meals_train = X_train['num_meals_not_home'].value_counts(normalize = True).to_frame('prob').reset_index()
meals_train.columns = ['k','prob']
meals_train.sort_values('k', inplace = True)
meals_train.reset_index(drop = True, inplace = True)
meals_train['data'] = 'real'
meals_train['split'] = 'train'

# Generate test distribution
meals_test = X_test['num_meals_not_home'].value_counts(normalize = True).to_frame('prob').reset_index()
meals_test.columns = ['k','prob']
meals_test.sort_values('k', inplace = True)
meals_test.reset_index(drop = True, inplace = True)
meals_test['data'] = 'real'
meals_test['split'] = 'test'

# Train-Test Split Visualization

In [ ]:
fig,ax = plt.subplots(figsize = (6,3))
g = sns.lineplot(
    data = meals_train,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '-',
    ax = ax
)
sns.lineplot(
    data = meals_test,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[2],
    linestyle = '-',
    ax = ax
)
g.set_xticks(range(22))
g.set_title('Train and Test Distributions From Dataset')
g.set_xlabel('Number of Times Eaten Out')
g.set_ylabel('Probability')
ax.legend(
    labels = ['Train', 'Test'],
    title = 'Data Source'
)
plt.grid()
plt.tight_layout()
plt.savefig('data_distributions.png', dpi = 250)
plt.show()

# Modeling

## Simulate Population Data

In [ ]:
k_sigma = [
    [0,0.2],
    [2.,1],
    [4,0.8],
    [5,0.4],
    [7.,0.4],
    [10.,0.1],
    [14,0.1],
    [21,0.1]
]

ns,ps,scale = generate_binomial_parameters(k_sigma,meals_train)
binomials = generate_distribution(ns,ps,scale)
binomials['data'] = 'simulated'
binomials['split'] = 'simulated'

In [ ]:
acc = pd.concat([
    binomials[['prob']],
    meals_train[['prob']],
    meals_test[['prob']]
], axis = 1)
acc.columns = ['sim','train','test']
train_error = acc[['sim','train']].min(axis = 1).sum()
test_error =  acc[['sim','test']].min(axis = 1).sum()
print('Accuracy')
print(F'Train H.I.: {np.round(train_error,3)}')
print(F'Test H.I.: {np.round(test_error,3)}')

In [ ]:
fig,ax = plt.subplots(figsize = (10,5))
g = sns.lineplot(
    data = binomials,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[1],
    linestyle = '--',
    ax = ax
)
sns.lineplot(
    data = meals_train,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '-',
    ax = ax
)
sns.lineplot(
    data = meals_test,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[2],
    linestyle = '-',
    ax = ax
)
g.set_xticks(range(22))
g.set_title('Real and Simulated Distributions')
g.set_xlabel('Number of Times Eaten Out')
g.set_ylabel('Probability')
ax.legend(
    labels = ['Simulation', 'Train', 'Test'],
    title = 'Data Source'
)
plt.grid()
plt.tight_layout()
# plt.savefig('base_simulation.png', dpi = 250)
plt.show()

In [ ]:
distribution_dict = {f'k{i}':f'$n_{i}={int(np.round(k_sigma[i-1][0],0))}$, $p_{i}={np.round(ps[i-1],3)}$' for i in range(1,8)}

melted_binomials = pd.melt(
    binomials[[
        'k',
        'k1',
        'k2',
        'k3',
        'k4',
        'k5',
        'k6',
        'k7'
    ]],
    id_vars = 'k'
)
melted_binomials['variable'] = melted_binomials['variable'].apply(lambda x: distribution_dict[x])

fig,ax = plt.subplots(figsize = (7,3))
g = sns.lineplot(
    data = melted_binomials,
    x = 'k',
    y = 'value',
    hue = 'variable',
)
g.set_xticks([k[0] for k in k_sigma])
g.set_title('Underlying Binomial Distributions')
g.set_xlabel('Number of Times Eaten Out')
g.set_ylabel('Probability')
plt.grid()
plt.legend(
    title = '$Binom(n_i,p_i)$',
    bbox_to_anchor=(1.05, 0.5),
    loc=6,
    borderaxespad=0.
)
plt.tight_layout()
plt.savefig('underlying_binomials.png', dpi = 250)

## Simulate Distributions Based on Single Variable Uncertainty

In [ ]:
# Select the feature you want to use for modification
# var = 'gender_male'
# var = 'race_Minority'
var = 'marital_status_Single'

og,zero,one = modify_probabilities(
    ps,
    ns,
    scale,
    k_sigma,
    meals_train,
    factor_scale = 0.15,
    factor_direction = -1,
    variable = var,
    new_scale = False
)

var_test = X_test.groupby(var)['num_meals_not_home'].value_counts(normalize = True).to_frame('prob').reset_index()
var_test.columns = [var,'k','prob']
# The printed output are the modified probabilities (f indicates 0 and m indicates 1)

In [ ]:
zero_acc = pd.merge(
    zero,
    var_test[var_test[var] == 0],
    on = 'k'
)
zero_error = zero_acc[['prob_x','prob_y']].min(axis = 1).sum()

one_acc = pd.merge(
    one,
    var_test[var_test[var] == 1],
    on = 'k'
)
one_error = one_acc[['prob_x','prob_y']].min(axis = 1).sum()

print('Accuracy')
print(f'Test 0 H.I.: {np.round(zero_error,3)}')
print(f'Test 1 H.I.: {np.round(one_error,3)}')

In [ ]:
fig,ax = plt.subplots(figsize = (10,5))
g = sns.lineplot(
    data = og,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[2],
    linestyle = '--',
    ax = ax
)

sns.lineplot(
    data = var_test[var_test[var] == 0],
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[1],
    linestyle = '-',
    ax = ax
)
sns.lineplot(
    data = zero,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[1],
    linestyle = '--',
    ax = ax
)

sns.lineplot(
    data = var_test[var_test[var] == 1],
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '-',
    ax = ax
)
sns.lineplot(
    data = one,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '--',
    ax = ax
)

g.set_xticks(range(22))
g.set_title('Modified Distributions')
g.set_xlabel('Number of Times Eaten Out')
g.set_ylabel('Probability')
ax.legend(
    labels = [
        'Simulation',
        f'Test: {feature_dict[var][0]}',
        f'Modified: {feature_dict[var][0]}',
        f'Test: {feature_dict[var][1]}',
        f'Modified: {feature_dict[var][1]}',
    ],
    title = 'Data Source'
)
plt.grid()
plt.tight_layout()
plt.savefig(f'{var}_modification.png', dpi = 250)
plt.show()

In [ ]:
# Same plot as above, but splitting into two subplots
fig,ax = plt.subplots(figsize = (8,5), nrows = 2)
sns.lineplot(
    data = og,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[2],
    linestyle = '--',
    ax = ax[0]
)

sns.lineplot(
    data = og,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[2],
    linestyle = '--',
    ax = ax[1]
)

sns.lineplot(
    data = var_test[var_test[var] == 0],
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[1],
    linestyle = '-',
    ax = ax[0]
)
sns.lineplot(
    data = zero,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[1],
    linestyle = '--',
    ax = ax[0]
)

sns.lineplot(
    data = var_test[var_test[var] == 1],
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '-',
    ax = ax[1]
)
sns.lineplot(
    data = one,
    x = 'k',
    y = 'prob',
    color = lirio_brand_colors[0],
    linestyle = '--',
    ax = ax[1]
)

ax[0].set_xlabel('Number of Times Eating Outside Home')
ax[1].set_xlabel('Number of Times Eating Outside Home')
ax[0].set_ylabel('Probability')
ax[1].set_ylabel('Probability')
plt.suptitle('Modified Distributions')

ax[0].legend(
    labels = [
        'Simulation',
        f'Test: {feature_dict[var][0]}',
        f'Modified: {feature_dict[var][0]}',
    ],
)
ax[1].legend(
    labels = [
        'Simulation',
        f'Test: {feature_dict[var][1]}',
        f'Modified: {feature_dict[var][1]}',
    ]
)

ax[0].set_xticks(range(22))
ax[1].set_xticks(range(22))
ax[0].grid()
ax[1].grid()
plt.tight_layout()
plt.savefig(f'{var}_modification_two_plots.png', dpi = 250)
# plt.show()